The neural network architecture is based on https://www.kaggle.com/artgor/movie-review-sentiment-analysis-eda-and-models 

Actually this model is an overkill for this competition and takes too long (8 hours!) to train and predict. This is what happened when you copied someone's code without knowing (or assumed wrongly) what you were actually doing.

Team yellow's simple feedforward neural network (https://www.kaggle.com/astraldawn/yellow-feedforward-neural-network) is much better in terms of speed and performance.

# Import

In [1]:
from datetime import datetime
start = datetime.now()

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from pathlib import Path
import json
import re
import sys
import warnings

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pickle
from sklearn.model_selection import train_test_split

In [5]:
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing import text, sequence
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Input, GlobalMaxPool1D, Conv1D, MaxPooling1D, GRU, concatenate, CuDNNGRU
from keras.layers import LSTM, CuDNNLSTM,  Bidirectional, SpatialDropout1D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
from keras import utils
from keras.utils import to_categorical
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.optimizers import Adam

keras.__version__, tf.__version__

Using TensorFlow backend.


('2.2.4-tf', '1.13.1')

In [6]:
VERSION = 'Deep_Learning_predictions'

In [7]:
DATA_DIR = Path('data')

BEAUTY_JSON = DATA_DIR / 'beauty_profile_train.json'
FASHION_JSON = DATA_DIR / 'fashion_profile_train.json'
MOBILE_JSON = DATA_DIR / 'mobile_profile_train.json'

BEAUTY_TRAIN_CSV = DATA_DIR / 'beauty_data_info_train_competition.csv'
FASHION_TRAIN_CSV = DATA_DIR / 'fashion_data_info_train_competition.csv'
MOBILE_TRAIN_CSV = DATA_DIR / 'mobile_data_info_train_competition.csv'

BEAUTY_TEST_CSV = DATA_DIR / 'beauty_data_info_val_competition.csv'
FASHION_TEST_CSV = DATA_DIR / 'fashion_data_info_val_competition.csv'
MOBILE_TEST_CSV = DATA_DIR / 'mobile_data_info_val_competition.csv'

In [8]:
with open(BEAUTY_JSON) as f:
     beauty_attribs = json.load(f)
        
with open(FASHION_JSON) as f:
     fashion_attribs = json.load(f)
        
with open(MOBILE_JSON) as f:
     mobile_attribs = json.load(f)

beauty_train_df = pd.read_csv(BEAUTY_TRAIN_CSV)
fashion_train_df = pd.read_csv(FASHION_TRAIN_CSV)
mobile_train_df = pd.read_csv(MOBILE_TRAIN_CSV)

beauty_test_df = pd.read_csv(BEAUTY_TEST_CSV)
fashion_test_df = pd.read_csv(FASHION_TEST_CSV)
mobile_test_df = pd.read_csv(MOBILE_TEST_CSV)

In [9]:
len(beauty_train_df), len(fashion_train_df), len(mobile_train_df)

(286583, 275142, 160330)

In [10]:
len(beauty_test_df), len(fashion_test_df), len(mobile_test_df)

(76545, 30135, 40417)

In [11]:
n_rows = len(beauty_test_df)*5 + len(fashion_test_df)*5 + len(mobile_test_df)*11 
print(n_rows)
assert n_rows == 977987, "Row numbers don't match!"

977987


# Hyperparameters

In [12]:
max_words_beauty = 29
max_words_fashion = 32
max_words_mobile = 27
batch_size = 32
epochs_beauty = [25, 25, 25, 25, 25]
epochs_fashion = [20, 20, 20, 20, 20]
epochs_mobile = [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
embed_size = 256

# Models

In [13]:
def build_model(max_features, max_len, num_classes, lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, 
                 kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    inp = Input(shape=(max_len,))
    x = Embedding(max_features, embed_size)(inp)
    x1 = SpatialDropout1D(spatial_dr)(x)

    x_gru = Bidirectional(CuDNNGRU(units, return_sequences=True))(x1)
    x_lstm = Bidirectional(CuDNNLSTM(units, return_sequences=True))(x1)

    x_conv1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x_conv1)
    max_pool1_gru = GlobalMaxPooling1D()(x_conv1)

    x_conv2 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool2_gru = GlobalAveragePooling1D()(x_conv2)
    max_pool2_gru = GlobalMaxPooling1D()(x_conv2)

    x_conv3 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x_conv3)
    max_pool1_lstm = GlobalMaxPooling1D()(x_conv3)

    x_conv4 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool2_lstm = GlobalAveragePooling1D()(x_conv4)
    max_pool2_lstm = GlobalMaxPooling1D()(x_conv4)

    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool2_gru, max_pool2_gru,
                     avg_pool1_lstm, max_pool1_lstm, avg_pool2_lstm, max_pool2_lstm])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu')(x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units/2), activation='relu')(x))
    x = Dense(units=num_classes, activation='softmax')(x)
    
    model = Model(inputs=inp, outputs=x)
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=["accuracy"])

    return model

In [14]:
def RNN_model(max_features, num_classes, maxlen):    
    model = build_model(max_features, maxlen, num_classes,
                         lr = 1e-4, lr_d = 0, units = 64, spatial_dr = 0.5, 
                         kernel_size1=4, kernel_size2=3, dense_units=32, dr=0.1, conv_size=32)
    return model

# Training

## Beauty

In [15]:
print(f'Total attributes for beauty products: {len(beauty_attribs)}')
print()

for i, attrib in enumerate(beauty_attribs, 1):
    print(f'Attribute_{i}: {attrib} ({len(beauty_attribs[attrib])} classes)')

all_beauty_titles = list(beauty_train_df.title)
beauty_tokenizer = Tokenizer()
beauty_tokenizer.fit_on_texts(all_beauty_titles)
tokenized_train_titles = beauty_tokenizer.texts_to_sequences(list(beauty_train_df.title))
print()
print(f'Total {len(beauty_tokenizer.word_index)} unique tokens.')

Total attributes for beauty products: 5

Attribute_1: Benefits (7 classes)
Attribute_2: Brand (401 classes)
Attribute_3: Colour_group (45 classes)
Attribute_4: Product_texture (9 classes)
Attribute_5: Skin_type (8 classes)

Total 29506 unique tokens.


In [16]:
map_beauty = []
for i, attrib in enumerate(beauty_attribs):
    num_classes = len(beauty_attribs[attrib])
    print(f'[Attrib_{i+1} {attrib}]: ({num_classes} classes)')
    beauty_attrib_train_df = beauty_train_df[['title', attrib]].dropna()
    titles = list(beauty_attrib_train_df.title)
    tokenized_train_titles = beauty_tokenizer.texts_to_sequences(titles)

    X = np.array(tokenized_train_titles)
    X = sequence.pad_sequences(X, padding='post', maxlen=max_words_beauty)
    y = to_categorical(beauty_attrib_train_df[attrib], num_classes)
    
    max_features = len(beauty_tokenizer.word_index) # how many unique words to use
    model = RNN_model(max_features, num_classes, max_words_beauty)
    
    file_path = f"{VERSION}_model_beauty_{attrib}.hdf5"
    
    hist = model.fit(X, y, 
                    batch_size=batch_size,
                    epochs=epochs_beauty[i],
                    verbose=0)
    model.save(file_path)
    print()

[Attrib_1 Benefits]: (7 classes)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

[Attrib_2 Brand]: (401 classes)

[Attrib_3 Colour_group]: (45 classes)

[Attrib_4 Product_texture]: (9 classes)

[Attrib_5 Skin_type]: (8 classes)



In [17]:
duration = datetime.now()-start
print('Total seconds:', duration.total_seconds())
print(f'Total minutes: {duration.total_seconds()/60:.2f}')
print(f'Total hour: {duration.total_seconds()/3600:.2f}')

Total seconds: 9768.316558
Total minutes: 162.81
Total hour: 2.71


## Fashion

In [18]:
print(f'Total attributes for fashion products: {len(fashion_attribs)}')
print()

for i, attrib in enumerate(fashion_attribs, 1):
    print(f'Attribute_{i}: {attrib} ({len(fashion_attribs[attrib])} classes)')

all_fashion_titles = list(fashion_train_df.title)
fashion_tokenizer = Tokenizer()
fashion_tokenizer.fit_on_texts(all_fashion_titles)
tokenized_train_titles = fashion_tokenizer.texts_to_sequences(list(fashion_train_df.title))
print()
print(f'Total {len(fashion_tokenizer.word_index)} unique tokens.')

Total attributes for fashion products: 5

Attribute_1: Pattern (20 classes)
Attribute_2: Collar Type (16 classes)
Attribute_3: Fashion Trend (11 classes)
Attribute_4: Clothing Material (19 classes)
Attribute_5: Sleeves (4 classes)

Total 45770 unique tokens.


In [19]:
map_fashion = []
for i, attrib in enumerate(fashion_attribs):
    num_classes = len(fashion_attribs[attrib])
    print(f'[Attrib_{i+1} {attrib}]: ({num_classes} classes)')
    fashion_attrib_train_df = fashion_train_df[['title', attrib]].dropna()
    titles = list(fashion_attrib_train_df.title)
    tokenized_train_titles = fashion_tokenizer.texts_to_sequences(titles)

    X = np.array(tokenized_train_titles)
    X = sequence.pad_sequences(X, padding='post', maxlen=max_words_fashion)
    y = to_categorical(fashion_attrib_train_df[attrib], num_classes)
    
    max_features = len(fashion_tokenizer.word_index) # how many unique words to use
    model = RNN_model(max_features, num_classes, max_words_fashion)
    
    file_path = f"{VERSION}_model_fashion_{attrib}.hdf5"
    
    hist = model.fit(X, y, 
                    batch_size=batch_size,
                    epochs=epochs_fashion[i],
                    verbose=0)
    model.save(file_path)
    print()

[Attrib_1 Pattern]: (20 classes)

[Attrib_2 Collar Type]: (16 classes)

[Attrib_3 Fashion Trend]: (11 classes)

[Attrib_4 Clothing Material]: (19 classes)

[Attrib_5 Sleeves]: (4 classes)



In [20]:
duration = datetime.now()-start
print('Total seconds:', duration.total_seconds())
print(f'Total minutes: {duration.total_seconds()/60:.2f}')
print(f'Total hour: {duration.total_seconds()/3600:.2f}')

Total seconds: 18575.284479
Total minutes: 309.59
Total hour: 5.16


## Mobile

In [21]:
print(f'Total attributes for mobile products: {len(mobile_attribs)}')
print()

for i, attrib in enumerate(mobile_attribs, 1):
    print(f'Attribute_{i}: {attrib} ({len(mobile_attribs[attrib])} classes)')

all_mobile_titles = list(mobile_train_df.title)
mobile_tokenizer = Tokenizer()
mobile_tokenizer.fit_on_texts(all_mobile_titles)
tokenized_train_titles = mobile_tokenizer.texts_to_sequences(list(mobile_train_df.title))
print()
print(f'Total {len(mobile_tokenizer.word_index)} unique tokens.')

Total attributes for mobile products: 11

Attribute_1: Operating System (7 classes)
Attribute_2: Features (7 classes)
Attribute_3: Network Connections (4 classes)
Attribute_4: Memory RAM (10 classes)
Attribute_5: Brand (56 classes)
Attribute_6: Warranty Period (14 classes)
Attribute_7: Storage Capacity (18 classes)
Attribute_8: Color Family (26 classes)
Attribute_9: Phone Model (2280 classes)
Attribute_10: Camera (15 classes)
Attribute_11: Phone Screen Size (6 classes)

Total 24529 unique tokens.


In [22]:
map_mobile = []
for i, attrib in enumerate(mobile_attribs):
    num_classes = len(mobile_attribs[attrib])
    print(f'[Attrib_{i+1} {attrib}]: ({num_classes} classes)')
    mobile_attrib_train_df = mobile_train_df[['title', attrib]].dropna()
    titles = list(mobile_attrib_train_df.title)
    tokenized_train_titles = mobile_tokenizer.texts_to_sequences(titles)

    X = np.array(tokenized_train_titles)
    X = sequence.pad_sequences(X, padding='post', maxlen=max_words_mobile)
    y = to_categorical(mobile_attrib_train_df[attrib], num_classes)
    
    max_features = len(mobile_tokenizer.word_index) # how many unique words to use
    model = RNN_model(max_features, num_classes, max_words_mobile)
    
    file_path = f"{VERSION}_model_mobile_{attrib}.hdf5"
    
    hist = model.fit(X, y, 
                    batch_size=batch_size,
                    epochs=epochs_mobile[i],
                    verbose=0)
    model.save(file_path)
    print()

[Attrib_1 Operating System]: (7 classes)

[Attrib_2 Features]: (7 classes)

[Attrib_3 Network Connections]: (4 classes)

[Attrib_4 Memory RAM]: (10 classes)

[Attrib_5 Brand]: (56 classes)

[Attrib_6 Warranty Period]: (14 classes)

[Attrib_7 Storage Capacity]: (18 classes)

[Attrib_8 Color Family]: (26 classes)

[Attrib_9 Phone Model]: (2280 classes)

[Attrib_10 Camera]: (15 classes)

[Attrib_11 Phone Screen Size]: (6 classes)



In [23]:
duration = datetime.now()-start
print('Total seconds:', duration.total_seconds())
print(f'Total minutes: {duration.total_seconds()/60:.2f}')
print(f'Total hour: {duration.total_seconds()/3600:.2f}')

Total seconds: 25631.789211
Total minutes: 427.20
Total hour: 7.12


# Inference

In [24]:
%%time
updated_beauty_title = list(beauty_test_df.title)
tokenized_test_titles = beauty_tokenizer.texts_to_sequences(updated_beauty_title)
X_test = np.array(tokenized_test_titles)
X_test = sequence.pad_sequences(X_test, padding='post', maxlen=max_words_beauty)

preds_all={}
for attrib in beauty_attribs:
    print(f'Attribute: {attrib}')
    print(f'Loading model...')
    model=load_model(f'{VERSION}_model_beauty_{attrib}.hdf5')
    print(f'Predicting...')
    pred_attrib=model.predict(X_test)
    print(f'Sorting predictions...')
    pred_list=[]
    for pred in pred_attrib:
        pred_list.append(pred.argsort()[-2:][::-1])
    preds_all[attrib]=np.array(pred_list.copy())
    print()
    
print(f'Saving to {VERSION}_pred_beauty.csv')
test_y_id=[] 
test_y_predictions=[]
for i, itemid in enumerate(beauty_test_df.itemid):
    for attrib in beauty_attribs:
        test_y_id.append(str(itemid) + f'_{attrib}')
        test_y_predictions.append(str(preds_all[attrib][i][0]) + ' ' + str(preds_all[attrib][i][1]))
        
beauty_result_df = pd.DataFrame(
    {'id': test_y_id, 'tagging': test_y_predictions},
    columns = ['id', 'tagging'])

beauty_result_df.to_csv(f'{VERSION}_pred_beauty.csv', index=False)

Attribute: Benefits
Loading model...
Predicting...
Sorting predictions...

Attribute: Brand
Loading model...
Predicting...
Sorting predictions...

Attribute: Colour_group
Loading model...
Predicting...
Sorting predictions...

Attribute: Product_texture
Loading model...
Predicting...
Sorting predictions...

Attribute: Skin_type
Loading model...
Predicting...
Sorting predictions...

Saving to Deep_Learning_predictions_pred_beauty.csv
CPU times: user 4min 17s, sys: 5.35 s, total: 4min 22s
Wall time: 4min 11s


In [25]:
%%time
updated_fashion_title = list(fashion_test_df.title)
tokenized_test_titles = fashion_tokenizer.texts_to_sequences(updated_fashion_title)
X_test = np.array(tokenized_test_titles)
X_test = sequence.pad_sequences(X_test, padding='post', maxlen=max_words_fashion)

preds_all={}
for attrib in fashion_attribs:
    print(f'Attribute: {attrib}')
    print(f'Loading model...')
    model=load_model(f'{VERSION}_model_fashion_{attrib}.hdf5')
    print(f'Predicting...')
    pred_attrib=model.predict(X_test)
    print(f'Sorting predictions...')
    pred_list=[]
    for pred in pred_attrib:
        pred_list.append(pred.argsort()[-2:][::-1])
    preds_all[attrib]=np.array(pred_list.copy())
    print()
    
print(f'Saving to {VERSION}_pred_fashion.csv')
test_y_id=[] 
test_y_predictions=[]
for i, itemid in enumerate(fashion_test_df.itemid):
    for attrib in fashion_attribs:
        test_y_id.append(str(itemid) + f'_{attrib}')
        test_y_predictions.append(str(preds_all[attrib][i][0]) + ' ' + str(preds_all[attrib][i][1]))
        
fashion_result_df = pd.DataFrame(
    {'id': test_y_id, 'tagging': test_y_predictions},
    columns = ['id', 'tagging'])

fashion_result_df.to_csv(f'{VERSION}_pred_fashion.csv', index=False)

Attribute: Pattern
Loading model...
Predicting...
Sorting predictions...

Attribute: Collar Type
Loading model...
Predicting...
Sorting predictions...

Attribute: Fashion Trend
Loading model...
Predicting...
Sorting predictions...

Attribute: Clothing Material
Loading model...
Predicting...
Sorting predictions...

Attribute: Sleeves
Loading model...
Predicting...
Sorting predictions...

Saving to Deep_Learning_predictions_pred_fashion.csv
CPU times: user 4min 32s, sys: 4.78 s, total: 4min 36s
Wall time: 4min 31s


In [26]:
%%time
updated_mobile_title = list(mobile_test_df.title)
tokenized_test_titles = mobile_tokenizer.texts_to_sequences(updated_mobile_title)
X_test = np.array(tokenized_test_titles)
X_test = sequence.pad_sequences(X_test, padding='post', maxlen=max_words_mobile)

preds_all={}
for attrib in mobile_attribs:
    print(f'Attribute: {attrib}')
    print(f'Loading model...')
    model=load_model(f'{VERSION}_model_mobile_{attrib}.hdf5')
    print(f'Predicting...')
    pred_attrib=model.predict(X_test)
    print(f'Sorting predictions...')
    pred_list=[]
    for pred in pred_attrib:
        pred_list.append(pred.argsort()[-2:][::-1])
    preds_all[attrib]=np.array(pred_list.copy())
    print()
    
print(f'Saving to {VERSION}_pred_mobile.csv')
test_y_id=[] 
test_y_predictions=[]
for i, itemid in enumerate(mobile_test_df.itemid):
    for attrib in mobile_attribs:
        test_y_id.append(str(itemid) + f'_{attrib}')
        test_y_predictions.append(str(preds_all[attrib][i][0]) + ' ' + str(preds_all[attrib][i][1]))
        
mobile_result_df = pd.DataFrame(
    {'id': test_y_id, 'tagging': test_y_predictions},
    columns = ['id', 'tagging'])

mobile_result_df.to_csv(f'{VERSION}_pred_mobile.csv', index=False)

Attribute: Operating System
Loading model...
Predicting...
Sorting predictions...

Attribute: Features
Loading model...
Predicting...
Sorting predictions...

Attribute: Network Connections
Loading model...
Predicting...
Sorting predictions...

Attribute: Memory RAM
Loading model...
Predicting...
Sorting predictions...

Attribute: Brand
Loading model...
Predicting...
Sorting predictions...

Attribute: Warranty Period
Loading model...
Predicting...
Sorting predictions...

Attribute: Storage Capacity
Loading model...
Predicting...
Sorting predictions...

Attribute: Color Family
Loading model...
Predicting...
Sorting predictions...

Attribute: Phone Model
Loading model...
Predicting...
Sorting predictions...

Attribute: Camera
Loading model...
Predicting...
Sorting predictions...

Attribute: Phone Screen Size
Loading model...
Predicting...
Sorting predictions...

Saving to Deep_Learning_predictions_pred_mobile.csv
CPU times: user 14min 44s, sys: 7.31 s, total: 14min 51s
Wall time: 14min 37

In [27]:
combined_df = pd.concat([beauty_result_df, fashion_result_df, mobile_result_df], axis=0)
combined_df.to_csv(f'{VERSION}.csv', index=None)

In [28]:
duration = datetime.now()-start
print('Total seconds:', duration.total_seconds())
print(f'Total minutes: {duration.total_seconds()/60:.2f}')
print(f'Total hour: {duration.total_seconds()/3600:.2f}')

Total seconds: 27034.357293
Total minutes: 450.57
Total hour: 7.51
